In [1]:
# https://zh.d2l.ai/chapter_preliminaries/autograd.html
# 逼近法就是积分（integral calculus）的起源
# 将拟合模型的任务分解为两个关键问题：
# 优化（optimization）：用模型拟合观测数据的过程；
# 泛化（generalization）：数学原理和实践者的智慧，能够指导我们生成出有效性超出用于训练的数据集本身的模型。

In [2]:
%matplotlib inline
import numpy as np
from matplotlib_inline import backend_inline
# pip install numpy==1.26.4 d2l
from d2l import torch as d2l
import torch

In [3]:
# f(x) = 3 x^2 - 4x
def f(x):
    return 3 * x ** 2 - 4 * x

# \frac{f(x+h)-f(x)}{h} 导数
def numerical_lim(f, x, h):
    return (f(x + h) - f(x)) / h

In [ ]:
h = 0.1
for i in range(5):
    print(f'h={h:.5f}, numerical limit={numerical_lim(f, 1, h):.5f}')
    h *= 0.1

In [5]:
def use_svg_display():  # @save
    """使用svg格式在Jupyter中显示绘图"""
    backend_inline.set_matplotlib_formats('svg')
    

def set_figsize(figsize=(3.5, 2.5)):  # @save
    """设置matplotlib的图表大小"""
    use_svg_display()
    d2l.plt.rcParams['figure.figsize'] = figsize
    
# @save
def set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
    """设置matplotlib的轴"""
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    if legend:
        axes.legend(legend)
    axes.grid()
    
# @save
def plot(X, Y=None, xlabel=None, ylabel=None, legend=None, xlim=None,
         ylim=None, xscale='linear', yscale='linear',
         fmts=('-', 'm--', 'g-.', 'r:'), figsize=(3.5, 2.5), axes=None):
    """绘制数据点"""
    if legend is None:
        legend = []

    set_figsize(figsize)
    axes = axes if axes else d2l.plt.gca()

    # 如果X有一个轴，输出True
    def has_one_axis(X):
        return (hasattr(X, "ndim") and X.ndim == 1 or isinstance(X, list)
                and not hasattr(X[0], "__len__"))

    if has_one_axis(X):
        X = [X]
    if Y is None:
        X, Y = [[]] * len(X), X
    elif has_one_axis(Y):
        Y = [Y]
    if len(X) != len(Y):
        X = X * len(Y)
    axes.cla()
    for x, y, fmt in zip(X, Y, fmts):
        if len(x):
            axes.plot(x, y, fmt)
        else:
            axes.plot(y, fmt)
    set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend)

In [ ]:
x = np.arange(0, 3, 0.1)
# 斜率
plot(x, [f(x), 2 * x - 3], 'x', 'f(x)',
     legend=['f(x)', 'Tangent line (x=1)'])

In [7]:
# 深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导
# 自动微分使系统能够随后反向传播梯度

In [ ]:
x = torch.arange(4.0)
x

In [9]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

In [ ]:
torch.dot(x, x)

In [ ]:
# y关于x的梯度
# (1 + 4 + 9) x 2 = 28
y = 2 * torch.dot(x, x)
y

In [ ]:
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad

In [ ]:
x.grad == 4 * x

In [ ]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y

In [ ]:
y.backward()
x.grad

In [ ]:
# 当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。 对于高阶和高维的y和x，求导的结果可以是一个高阶张量。

In [ ]:
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

In [ ]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
# 关于x的偏导数
z.sum().backward()
x.grad == u

In [ ]:
# 在y上调用反向传播， 得到y=x*x关于的x的导数，即2*x
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

In [21]:
# 使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c


a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [ ]:
a.grad == d / a